<a href="https://colab.research.google.com/github/marymlucas/scrap/blob/main/reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploring how to define under-reporting

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DSCI521_Project/project/data/timeseries_data.csv')

In [4]:
data.head()

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,previous_day_admission_influenza_confirmed,previous_day_admission_influenza_confirmed_coverage,previous_day_deaths_covid_and_influenza,previous_day_deaths_covid_and_influenza_coverage,previous_day_deaths_influenza,previous_day_deaths_influenza_coverage,total_patients_hospitalized_confirmed_influenza,total_patients_hospitalized_confirmed_influenza_and_covid,total_patients_hospitalized_confirmed_influenza_and_covid_coverage,total_patients_hospitalized_confirmed_influenza_coverage
0,VT,2020/10/18,1,15,1,1,15,1,0.0,16,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
1,VT,2020/10/17,1,15,1,1,15,1,0.0,16,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
2,ND,2020/10/16,20,28,1,20,28,1,8.0,48,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
3,ND,2020/10/15,20,28,1,21,27,1,8.0,48,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0
4,ND,2020/10/14,21,27,1,22,26,1,8.0,48,...,NaN,0,NaN,0,NaN,0,NaN,NaN,0,0


According to the data dictionary the columns with '_coverage' in the name give us the number of hospitals that reported the corresponding number

In [10]:
coverage_cols = [col for col in data.columns if 'coverage' in col]
coverage_cols

['hospital_onset_covid_coverage',
 'inpatient_beds_coverage',
 'inpatient_beds_used_coverage',
 'inpatient_beds_used_covid_coverage',
 'previous_day_admission_adult_covid_confirmed_coverage',
 'previous_day_admission_adult_covid_suspected_coverage',
 'previous_day_admission_pediatric_covid_confirmed_coverage',
 'previous_day_admission_pediatric_covid_suspected_coverage',
 'staffed_adult_icu_bed_occupancy_coverage',
 'staffed_icu_adult_patients_confirmed_and_suspected_covid_coverage',
 'staffed_icu_adult_patients_confirmed_covid_coverage',
 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_coverage',
 'total_adult_patients_hospitalized_confirmed_covid_coverage',
 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_coverage',
 'total_pediatric_patients_hospitalized_confirmed_covid_coverage',
 'total_staffed_adult_icu_beds_coverage',
 'inpatient_beds_utilization_coverage',
 'percent_of_inpatients_with_covid_coverage',
 'inpatient_bed_covid_utilization_cover

## Get total number of hospitals per state
The most current list of hospital numbers was obtained from https://www.kff.org/other/state-indicator/total-hospitals which reports data from the American Hospital Association (1999 - 2019 AHA Annual Survey) 

It's important to note the disclaimer that "Data are for community hospitals, which represent 85% of all hospitals" where community hospitals are defined as "all nonfederal, short-term general, and specialty hospitals whose facilities and services are available to the public."


In [46]:
# csv file with hospital counts by state 
hosp_counts_by_state = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DSCI521_Project/project/data/hospitals_by_state.csv')

In [47]:
hosp_counts_by_state.head()

,Location,Total Hospitals
0,United States,5141
1,Alabama,101
2,Alaska,20
3,Arizona,80
4,Arkansas,89


In [48]:
hosp_data = hosp_counts_by_state[1:]

In [49]:
hosp_data.head()

,Location,Total Hospitals
1,Alabama,101
2,Alaska,20
3,Arizona,80
4,Arkansas,89
5,California,359


In [50]:
# map state name to state abbrv
# dictionary credit: https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

hosp_data['state'] = hosp_data.Location.map(us_state_to_abbrev)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [51]:
hosp_data.head()

,Location,Total Hospitals,state
1,Alabama,101,AL
2,Alaska,20,AK
3,Arizona,80,AZ
4,Arkansas,89,AR
5,California,359,CA


In [53]:
hosp_data = hosp_data[['state', 'Total Hospitals']]

In [55]:
hosp_data = hosp_data.rename({'Total Hospitals': 'total_hospitals'}, axis=1)

In [56]:
hosp_data.head()

,state,total_hospitals
1,AL,101
2,AK,20
3,AZ,80
4,AR,89
5,CA,359


## Merge the two dataframes to add a total_hospitals feature to the data


In [58]:
hosp_data.shape

(51, 2)

In [59]:
data.shape

(39105, 117)

In [60]:
data_new = pd.merge(data, hosp_data, how="left", on='state')
data_new.head()

,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,previous_day_admission_influenza_confirmed_coverage,previous_day_deaths_covid_and_influenza,previous_day_deaths_covid_and_influenza_coverage,previous_day_deaths_influenza,previous_day_deaths_influenza_coverage,total_patients_hospitalized_confirmed_influenza,total_patients_hospitalized_confirmed_influenza_and_covid,total_patients_hospitalized_confirmed_influenza_and_covid_coverage,total_patients_hospitalized_confirmed_influenza_coverage,total_hospitals
0,VT,2020/10/18,1,15,1,1,15,1,0.0,16,...,0,NaN,0,NaN,0,NaN,NaN,0,0,14.0
1,VT,2020/10/17,1,15,1,1,15,1,0.0,16,...,0,NaN,0,NaN,0,NaN,NaN,0,0,14.0
2,ND,2020/10/16,20,28,1,20,28,1,8.0,48,...,0,NaN,0,NaN,0,NaN,NaN,0,0,40.0
3,ND,2020/10/15,20,28,1,21,27,1,8.0,48,...,0,NaN,0,NaN,0,NaN,NaN,0,0,40.0
4,ND,2020/10/14,21,27,1,22,26,1,8.0,48,...,0,NaN,0,NaN,0,NaN,NaN,0,0,40.0


In [62]:
data_new[['state', 'staffed_adult_icu_bed_occupancy_coverage', 'total_staffed_adult_icu_beds_coverage', 'adult_icu_bed_utilization_coverage', 'total_hospitals']]

,state,staffed_adult_icu_bed_occupancy_coverage,total_staffed_adult_icu_beds_coverage,adult_icu_bed_utilization_coverage,total_hospitals
0,VT,17,17,17.0,14.0
1,VT,17,17,17.0,14.0
2,ND,49,49,49.0,40.0
3,ND,49,49,49.0,40.0
4,ND,49,49,49.0,40.0
...,...,...,...,...,...
39100,TX,0,0,NaN,512.0
39101,HI,0,0,NaN,22.0
39102,HI,0,0,NaN,22.0
39103,NV,0,0,NaN,43.0


In [87]:
# proportion of hospitals reporting each day
data_new['percent_hosps_reporting'] = data_new['adult_icu_bed_utilization_coverage']/data_new['total_hospitals']

In [64]:
data_new[['state', 'adult_icu_bed_utilization_coverage', 'total_hospitals', 'percent_hosps_reporting']]

,state,adult_icu_bed_utilization_coverage,total_hospitals,percent_hosps_reporting
0,VT,17.0,14.0,1.214286
1,VT,17.0,14.0,1.214286
2,ND,49.0,40.0,1.225000
3,ND,49.0,40.0,1.225000
4,ND,49.0,40.0,1.225000
...,...,...,...,...
39100,TX,NaN,512.0,NaN
39101,HI,NaN,22.0,NaN
39102,HI,NaN,22.0,NaN
39103,NV,NaN,43.0,NaN


It makes sense that for some states like VT we have greater than 100% reporting since the number of hospitals reporting is greater than the reported number of hospitals in the state. See disclaimer in the section about the total number of hospitals data. 

In [72]:
data_new.sort_values(by=['percent_hosps_reporting'])


,state,date,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,...,previous_day_deaths_covid_and_influenza,previous_day_deaths_covid_and_influenza_coverage,previous_day_deaths_influenza,previous_day_deaths_influenza_coverage,total_patients_hospitalized_confirmed_influenza,total_patients_hospitalized_confirmed_influenza_and_covid,total_patients_hospitalized_confirmed_influenza_and_covid_coverage,total_patients_hospitalized_confirmed_influenza_coverage,total_hospitals,percent_hosps_reporting
7517,NY,2020/05/02,0,1,146,0,1,146,68.0,143,...,0.0,1,0.0,1,0.0,0.0,1,1,163.0,0.006135
4871,NY,2020/06/25,0,1,140,0,1,140,18.0,136,...,0.0,1,0.0,1,0.0,0.0,1,1,163.0,0.006135
4869,NY,2020/06/28,0,1,132,0,1,132,16.0,129,...,0.0,1,0.0,1,0.0,0.0,1,1,163.0,0.006135
7746,NY,2020/05/08,0,1,145,0,1,145,70.0,141,...,0.0,1,0.0,1,0.0,0.0,1,1,163.0,0.006135
4823,NY,2020/06/30,0,1,137,0,1,137,16.0,134,...,0.0,1,0.0,1,0.0,0.0,1,1,163.0,0.006135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39100,TX,2020/02/02,0,0,3,0,0,3,0.0,1,...,NaN,0,NaN,0,NaN,NaN,0,0,512.0,NaN
39101,HI,2020/01/05,0,0,1,0,0,1,0.0,1,...,NaN,0,NaN,0,NaN,NaN,0,0,22.0,NaN
39102,HI,2020/01/03,0,0,1,0,0,1,0.0,1,...,NaN,0,NaN,0,NaN,NaN,0,0,22.0,NaN
39103,NV,2020/02/12,0,0,1,0,0,1,0.0,1,...,NaN,0,NaN,0,NaN,NaN,0,0,43.0,NaN


In [91]:
# how many rows are there by state
data.groupby('state').size().sort_values(ascending=False)

state
MT    790
TX    790
AL    790
NC    790
IN    790
HI    790
MN    790
NV    772
KS    759
IL    742
MS    741
WV    740
MO    737
OR    736
PR    735
LA    735
CA    735
WA    732
OK    730
OH    730
NJ    730
PA    730
NE    730
ND    730
WI    730
WY    730
MI    730
MD    730
KY    730
IA    730
GA    730
ME    730
VA    729
AZ    729
SC    729
RI    728
AR    725
FL    719
ID    718
NY    717
TN    717
NM    717
CO    717
VT    714
CT    713
UT    711
AK    708
SD    706
DE    704
NH    704
MA    704
DC    703
VI    692
AS    187
dtype: int64

So perhaps we could first consider dropping American Samoa (AS) and Virgin Islands (VI) from our analysis?

In [75]:
# set different thresholds for definition of under-reporting

under_reporting_1 = data_new[data_new["percent_hosps_reporting"] < 0.75] #strictest
under_reporting_2 = data_new[data_new["percent_hosps_reporting"] < 0.5]
under_reporting_3 = data_new[data_new["percent_hosps_reporting"] < 0.25]
under_reporting_4 = data_new[data_new["percent_hosps_reporting"] < 0.1] # most generous


In [92]:
# occurences of underreporting
under_reporting_1.groupby('state').size().sort_values(ascending=False).head(10)

state
NY    135
MS    115
RI    112
LA     94
WA     91
HI     90
KS     86
IL     79
AK     78
GA     73
dtype: int64

In [93]:
under_reporting_2.groupby('state').size().sort_values(ascending=False).head(10)

state
RI    111
NY     96
AK     62
MT     55
MS     38
HI     37
KS     35
VT     33
SD     31
NJ     26
dtype: int64

In [94]:
under_reporting_3.groupby('state').size().sort_values(ascending=False).head(10)

state
RI    104
NY     84
MS     27
NJ     25
MT     18
ME     16
NE     13
KY     11
DC     10
MI     10
dtype: int64

In [95]:
under_reporting_4.groupby('state').size().sort_values(ascending=False).head(10)

state
RI    104
NY     84
MS     24
ME     14
NE     12
MI     10
KY      8
LA      8
AL      7
CT      6
dtype: int64